In [2]:
import pandas as pd
import numpy as np
import xlwings as xw 

In [3]:
def clean_tollCost_df(df):
    df = df.loc[df['LPN/Tag number'].str.isnumeric()]
    df['tag_serial_number'] = pd.to_numeric(df['LPN/Tag number'])
    df['tag_serial_number'] = pd.to_numeric(df['tag_serial_number'])
    df['Trip Cost'] = df['Trip Cost'].str.replace('$', '').astype(float)
    df = df.rename(columns={'Start Date': 'Date', "Trip Cost" : "trip_cost"})
    df.Date = pd.to_datetime(df.Date, format="%d/%m/%Y %H:%M")
    df = df[['Date', 'Details', 'trip_cost', 'tag_serial_number']]
    return df
def clean_roster_df(df):
    df.Primary_truck = df.Primary_truck.str.replace(" ", "")
    df.Alternative_Truck = df.Alternative_Truck.str.replace(" ", "")
    df = df.fillna(0)
    return df


def pri_truck_toll_cost(df):
    emp_id = df.Primary_employeeID
    sal = sal_per_shift.loc[sal_per_shift.Employee_ID == emp_id].per_shift_pay.values
    if len(sal) != 0:
       return sal[0]
    else:
        return 0

def sec_truck_toll_cost(df):
    Primary_truck_plate = df.Primary_truck
    sal = sal_per_shift.loc[sal_per_shift.Employee_ID == Primary_truck_plate].per_shift_pay.values
    if len(sal) != 0:
       return sal[0]
    else:
        return 0

In [9]:
PATH_TAG_ID = "D:\\Run Analysis\\blob_storage\\toll_id.csv"
PATH_TOLL = "D:\\Run Analysis\\blob_storage\\truck_toll\\40th_2021.csv"
PATH_ROS = "D:\\Run Analysis\\blob_storage\\roster\\40th_2021.xlsx"

tagid_df = pd.read_csv(PATH_TAG_ID).dropna(subset=["number_plate"])
tollCost_df = pd.read_csv(PATH_TOLL)
roster_df = pd.read_excel(PATH_ROS)

tollCost_df = clean_tollCost_df(tollCost_df)
roster_df = clean_roster_df(roster_df)

s1 = roster_df.Primary_truck.value_counts()
s2 = roster_df.Alternative_Truck.value_counts()
s3 = pd.concat([s1, s2]).reset_index().groupby('index').sum().iloc[1:].rename(columns={0 : "runs"})

<ipython-input-3-a2535915eb30>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tag_serial_number'] = pd.to_numeric(df['LPN/Tag number'])
<ipython-input-3-a2535915eb30>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tag_serial_number'] = pd.to_numeric(df['tag_serial_number'])
<ipython-input-3-a2535915eb30>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when r

In [10]:
# pd.concat([s1, s2]).sort_values(axis=1)  .sort_values(by='index') 
s3



,runs
index,
BL82AG,8
BY46TO,2
CF48ED,2
CF71JM,4
CG94KY,1
CH32FV,5
CJ65PS,9
CL02TB,7
CL98LU,5


In [31]:


tollcost_by_truck_df = pd.merge(tollCost_df, tagid_df, how='left', on='tag_serial_number').groupby('number_plate')['trip_cost'].sum().reset_index()        
# pd.merge(tollcost_by_truck_df, )

df_5 = pd.merge(tollcost_by_truck_df, s3, how='left', left_on='number_plate', right_on='index').dropna()
df_5['per_cost'] = df_5.trip_cost / df_5.runs

df_5 = df_5[['number_plate', 'per_cost']]
# tollcost_by_truck_df
df_5_dict = df_5.set_index('number_plate')['per_cost'].to_dict()



roster_df['pri_truck_toll_cost'] = roster_df.apply(lambda x : df_5_dict[x['Primary_truck']] if x['Primary_truck'] in df_5_dict else 0, axis=1)


roster_df['sec_truck_toll_cost'] = roster_df.apply(lambda x : df_5_dict[x['Alternative_Truck']] if x['Alternative_Truck'] in df_5_dict else 0, axis=1)
roster_df['toll_cost'] = roster_df['pri_truck_toll_cost'] + roster_df['sec_truck_toll_cost']

In [35]:
roster_df = roster_df[['Date', 'toll_cost', 'Primary_route', 'Satellite_Route_1','Satellite_Route_2']]

def per_route_cost(df):
    count = 0 
    p_route = df.Primary_route
    route_1 = df.Satellite_Route_1
    route_2 = df.Satellite_Route_2
    if p_route != 0:
        count = count + 1
    if route_1 != 0:
        count = count + 1
    if route_2 != 0:
        routes = route_2.split("/")
        count = count + len(routes)
    
    cost = df.toll_cost / count
    return cost, count

roster_df[['per_run_toll', "count_of"]] = roster_df.apply(per_route_cost, axis=1, result_type='expand') 



In [39]:
roster_df[roster_df.Satellite_Route_1.ne(0)]

,Date,toll_cost,Primary_route,Satellite_Route_1,Satellite_Route_2,per_run_toll,count_of
19,2021-10-06,40.042500,HYG,RL5,RL6/RLP,10.010625,4.0
20,2021-10-12,40.042500,RL5,RL6,0,20.021250,2.0
53,2021-10-12,5.495000,NIWPR2,CBK,0,2.747500,2.0
58,2021-10-08,14.772000,NIWPR2,CBK,0,7.386000,2.0
83,2021-10-07,19.044000,NIWPR2,CBK,0,9.522000,2.0
86,2021-10-06,5.495000,NIWPR2,CBK,0,2.747500,2.0
89,2021-10-06,25.859028,BR2,RL9,0,12.929514,2.0
91,2021-10-12,17.987778,BR2,RL9,0,8.993889,2.0
94,2021-10-08,25.859028,RL9,BR2,0,12.929514,2.0
95,2021-10-09,25.859028,BR2,RL9,0,12.929514,2.0
